In [23]:
%%writefile teste.txt
31 5
2 4 5 3 2 
5 9 5
17 20 4
13 16 4
16 18 1
3 6 2
6 8 1
19 21 4
23 3 5
20 23 4
21 23 1
15 17 3
7 9 4
1 4 1
10 13 4
23 2 5
3 6 1
8 11 2
12 14 4
17 21 5
17 20 4
12 17 4
13 16 5
21 2 4
20 0 4
17 20 3
10 12 3
3 4 2
0 2 5
20 0 4
7 10 4
13 17 2

Overwriting teste.txt


In [20]:
%%writefile teste-exaustiva-thrust.cu
#include <iostream>
#include <iomanip>
#include <cstdlib>
#include <chrono>
#include <cstdlib>
#include <algorithm>
#include <cmath>
#include <random>
// Importações do thrust
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/functional.h>
#include <thrust/copy.h>

using namespace std;


struct analisa_configuracao {
    int n_filmes;
    int n_categorias;
    int *start_time;
    int *end_time;
    int *cat_id;
    int *categorias;

    analisa_configuracao(int _n_filmes,
                         int _n_categorias,
                         int* _start_time,
                         int* _end_time,
                         int* _cat_id,
                         int* _categorias
                        ): n_filmes(_n_filmes), 
                           n_categorias(_n_categorias),
                           start_time(_start_time),
                           end_time(_end_time),
                           cat_id(_cat_id),
                           categorias(_categorias)
                        {};

    __device__
    int operator()(const int& x){
        int disponibilidade[24];
        int copy_categorias[10];

        for (int k = 0; k < 24; k++){
            disponibilidade[k] = 0;
        }

        for (int l = 0; l < n_categorias; l++){
            copy_categorias[l] = categorias[l];
        }

        int max_count = 0;

        for (int i = 0; i < n_filmes; i++){
            if (x & (1<<i)){
                if (copy_categorias[cat_id[i] - 1] <= 0) return -1;
                for (int j = start_time[i]; j < end_time[i]; j++){
                    if (disponibilidade[j] == 1) return -1;
                    disponibilidade[j] = 1;
                }
                copy_categorias[cat_id[i] - 1]--;
                max_count++;
            }
        }
        return max_count;
    }
};


int main(){
    int n_filmes, n_categorias;

    cin >> n_filmes >> n_categorias;

    thrust::host_vector<int> categorias(n_categorias);
    thrust::host_vector<int> start_time(n_filmes);
    thrust::host_vector<int> end_time(n_filmes);
    thrust::host_vector<int> cat_id(n_filmes);

    thrust::host_vector<int> result(1);

    for (int i = 0; i < n_categorias; i++){
        cin >> categorias[i];
    }

    int n_start_time, n_end_time;

    for (int i = 0; i < n_filmes; i++){
        cin >> n_start_time >> n_end_time;
        if (n_end_time == 0) n_end_time = 24;
        if (n_start_time < 0) n_start_time = 0;
        if (n_end_time < 0) n_end_time = 0;
        if (n_end_time < n_start_time) n_end_time = 24;
        start_time[i] = n_start_time;
        end_time[i] = n_end_time;
        cin >> cat_id[i];
    }

    cout << "Carregou as entradas" << endl;

    thrust::device_vector<int> device_categorias = categorias;
    thrust::device_vector<int> device_start_time = start_time;
    thrust::device_vector<int> device_end_time = end_time;
    thrust::device_vector<int> device_cat_id = cat_id;

    cout << "Copiou para o device" << endl;

    thrust::device_vector<int> device_configuration(pow(2, n_filmes));
    thrust::sequence(device_configuration.begin(), device_configuration.end());

    cout << "Criou o vetor de configurações" << endl;


    thrust::transform(device_configuration.begin(), device_configuration.end(), device_configuration.begin(), 
                      analisa_configuracao(n_filmes, 
                                           n_categorias,
                                           thrust::raw_pointer_cast(device_start_time.data()), 
                                           thrust::raw_pointer_cast(device_end_time.data()),
                                           thrust::raw_pointer_cast(device_cat_id.data()),
                                           thrust::raw_pointer_cast(device_categorias.data())
                                          )
                     );

    cout << "Aplicou o Transform" << endl;

    thrust::device_vector<int> device_result(1);

    device_result[0] = *thrust::max_element(device_configuration.begin(), device_configuration.end());
    thrust::copy(device_result.begin(), device_result.end(), result.begin());

    cout << "Max Filmes: " << result[0] << endl;

    return 0;
}


Overwriting teste-exaustiva-thrust.cu


In [21]:
!nvcc teste-exaustiva-thrust.cu -o teste-exaustiva-thrust

In [24]:
!./teste-exaustiva-thrust < teste.txt

Carregou as entradas
Copiou para o device
Criou o vetor de configurações
Aplicou o Transform
Max Filmes: 9


In [8]:
%%writefile busca-exaustiva.cpp
#include <iostream>
#include <iomanip>      // std::setprecision
#include <cmath>        // std::pow
#include <vector>       // std::vector
#include <algorithm>    // std::sort
#include <bitset>

//==============================================================================
// Structs
//==============================================================================

struct Filme{
    int id;
    int inicio;
    int fim;
    int categoria;
};

struct Proc_Filme{
    int id;
    std::bitset<24> horario;
    int categoria;
};

struct Categoria{
    int id;
    int capacidade;
};

//==============================================================================
// Funtions
//==============================================================================

std::vector<Proc_Filme> get_horario(std::vector<Filme> Filmes, int N){
    std::vector<Proc_Filme> filmes_processados(N);

    for (int i = 0; i < N; i++){
        int inicio = Filmes[i].inicio;
        int fim = Filmes[i].fim;

        std::bitset<24> horario;

        // Produzir horário em bitset
        if (inicio > fim) {
            for (int i = inicio; i < 24; i++) horario.set(i);
            for (int i = 0; i < fim; i++) horario.set(i);
        }
        else if(inicio == fim) {
            horario.set(inicio);
        }
        else {
            for (int i = inicio; i < fim; i++) horario.set(i);
        }

        Proc_Filme proc_filme;
        proc_filme.id = Filmes[i].id;
        proc_filme.horario = horario;
        proc_filme.categoria = Filmes[i].categoria;
        filmes_processados[i] = proc_filme;
    }

    return filmes_processados;
};

//==============================================================================
// Main
//==============================================================================
int main(){
    int n_filmes, n_categorias;
    std::vector<Filme> filmes;
    std::vector<Categoria> categorias;

    std::cin >> n_filmes >> n_categorias;

    for(int i = 1; i <= n_categorias; i++){
        Categoria categoria;
        categoria.id = i;
        std::cin >> categoria.capacidade;
        categorias.push_back(categoria);
    }

    for(int i = 0; i < n_filmes; i++){
        Filme filme;
        filme.id = i;
        std::cin >> filme.inicio >> filme.fim >> filme.categoria;
        if (filme.fim == 0) filme.fim = 24;
        if (filme.inicio < 0) filme.inicio = 0;
        if (filme.fim < 0) filme.fim = 0;
        if (filme.fim < filme.inicio) filme.fim = 24;
        filmes.push_back(filme);
    };

    n_filmes = filmes.size();

    std::vector<Proc_Filme> filmes_processados = get_horario(filmes, n_filmes);

    std::vector<std::bitset<64>> possibilidades;

    for(long int i = 0; i < pow(2, n_filmes); i++){
        std::vector<Categoria> copy_categorias = categorias;
        std::bitset<24> maratona;
        std::bitset<64> x(i);

        for (int j = 0; j < n_filmes; j++){
            if (x[j] == 1){
                std::bitset<24> disponivel = maratona & filmes_processados[j].horario;
                if(disponivel != 0) break;
                if(copy_categorias[filmes_processados[j].categoria-1].capacidade == 0) break;
                maratona = maratona | filmes_processados[j].horario;
                copy_categorias[filmes_processados[j].categoria-1].capacidade--;
            }
            if (j == n_filmes-1) possibilidades.push_back(x);
        }
    }

    int n_possibilidades = possibilidades.size();
    std::vector<int> n_filmes_possibilidades(n_possibilidades);
    for(int i = 0; i < n_possibilidades; i++){
        for (int j = 0; j < 64; j++){
            if (possibilidades[i][j] == 1) n_filmes_possibilidades[i]++;
        }
    }

    int max = *std::max_element(n_filmes_possibilidades.begin(), n_filmes_possibilidades.end());
    std::cout << max << std::endl;

    return 0;
}

Writing busca-exaustiva.cpp


In [9]:
!g++ -g -O3 -Wall busca-exaustiva.cpp -o busca-exaustiva

In [10]:
!./busca-exaustiva < teste.txt

9
